In [1]:
from sodapy import Socrata
import pandas as pd
import json

# City of Los Angeles Building Permits
-[Link](https://dev.socrata.com/foundry/data.lacity.org/nbyu-2ha9)

## A.1 API key

In [2]:
api_file = open('api_key.txt')
api_code = "".join(api_file.readlines())

## A.2 Download client

In [3]:
client = Socrata('data.lacity.org', api_code)

## B. Meta data

In [43]:
metadata = client.get_metadata('nbyu-2ha9')
[x['name'] for x in metadata['columns']][:5]

['Assessor Book', 'Assessor Page', 'Assessor Parcel', 'Tract', 'Block']

In [44]:
meta_amount = [x for x in metadata['columns'] if x['name'] == 'Status Date'][0]
meta_amount

{'id': 422923858,
 'name': 'Status Date',
 'dataTypeName': 'calendar_date',
 'description': 'Permit status date.',
 'fieldName': 'status_date',
 'position': 10,
 'renderTypeName': 'calendar_date',
 'tableColumnId': 31067931,
 'width': 223,
 'format': {'view': 'date_time', 'align': 'left'}}

# 1. Download data
## 1.1 Get raw 

In [29]:
results = client.get('nbyu-2ha9',
                     limit =200000,
                     where = f"permit_type='Bldg-New'"
                    )
#results

## 1.2 Convert to dataframe

In [32]:
df = pd.DataFrame(results)
# Datetime
df.issue_date = pd.to_datetime(df.issue_date)
# Set index
df.set_index('issue_date', inplace=True)
# Sort index
df = df.sort_index()
df.head(2)

,zip_code,address_end,floor_area_l_a_building_code_definition,work_description,reference_old_permit,census_tract,permit_category,latest_status,initiating_office,assessor_parcel,...,principal_last_name,contractor_address,address_fraction_start,address_fraction_end,block,applicant_business_name,applicant_address_2,occupancy,project_number,suffix_direction
issue_date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,90045,7270,463937,CONSTRUCT NEW MIXED-USE OF RETAILS AND 4-STORY...,13LA11079,2766.04,Plan Check,CofO Issued,METRO,042,...,BERNARDS,555 FIRST STREET,NaN,NaN,NaN,VTBS,NaN,NaN,NaN,NaN
2013-01-02,90048,132,4501,New SFD with attached garage <Fire sprinklers ...,13VN50614,2146.00,Plan Check,CofO Issued,VAN NUYS,020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2.1 Show dtypes

In [33]:
len(df)

33745

In [34]:
df.dtypes[:5]

zip_code                                   object
address_end                                object
floor_area_l_a_building_code_definition    object
work_description                           object
reference_old_permit                       object
dtype: object

## 1.3 Show data for dates

- What is status_date?

In [37]:
df.loc[:, ['permit_type',
           'work_description',
           'applicant_last_name', 
           'license_expiration_date', 
           'status_date']].head(3)

,permit_type,work_description,applicant_last_name,license_expiration_date,status_date
issue_date,,,,,
2013-01-02,Bldg-New,CONSTRUCT NEW MIXED-USE OF RETAILS AND 4-STORY...,LEUS,2013-03-31T00:00:00.000,2015-07-14T00:00:00.000
2013-01-02,Bldg-New,New SFD with attached garage <Fire sprinklers ...,GARCIA,NaN,2013-10-10T00:00:00.000
2013-01-02,Bldg-New,"CONSTRUCT A 4-STORY, (8 UNIT) APARTMENT BUILD...",NaN,2014-01-01T00:00:00.000,2014-02-25T00:00:00.000


## 1.4 Show permit approval by year

In [36]:
grouper = df.groupby([pd.Grouper(freq='A'), 'permit_sub_type'])
grouper['permit_sub_type'].count().unstack()

permit_sub_type,1 or 2 Family Dwelling,Apartment,Commercial
issue_date,,,
2013-12-31,2058,183,260
2014-12-31,2667,254,319
2015-12-31,3094,225,278
2016-12-31,3139,269,260
2017-12-31,3724,255,346
2018-12-31,4004,311,336
2019-12-31,3927,277,337
2020-12-31,2680,176,249
2021-12-31,3173,209,181
